In [1]:
import nltk
import numpy as np
import collections
import re
import pandas as pd
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import SnowballStemmer
from csv_helper import CSVHelper
nltk.download('stopwords')
nltk.download('wordnet')

data = CSVHelper.load_csv("Tweets_2016London.csv")

[nltk_data] Downloading package stopwords to C:\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
starting for loop!


In [2]:
def tokenize(tweets):
    tknzr = TweetTokenizer()
    tokenized_tweets = []
    for tweet in tweets:
        tokenized_tweets.append(tknzr.tokenize(tweet))
    return tokenized_tweets

def remove_stopwords(tweets):
    stopwords =  nltk.corpus.stopwords.words('english')
    tweets_nostop = []
    for tweet in tweets:
        tweet_nostop = [w.lower() for w in tweet if w.lower() not in stopwords]
        tweets_nostop.append(tweet_nostop)
    return tweets_nostop

def filter_noise(tweets):
    tweets_filtered = []
    for tweet in tweets:
        tweet_filtered = [re.sub("[^a-zA-Z0-9]",'', w) for w in tweet if re.sub("[^a-zA-Z0-9]",'', w)!= '']
        tweets_filtered.append(tweet_filtered)
    return tweets_filtered

def remove_url(tweets):
    tweets_filtered = []
    for tweet in tweets:
        tweet_filtered = [re.sub(r'^https?:\/\/.*[\r\n]*', '', w) for w in tweet]
        tweets_filtered.append(tweet_filtered)
    return tweets_filtered

#we decided to use a lemmatizer instead of stemming 
def my_stem(tweets):
    lmt = WordNetLemmatizer()
    ps = PorterStemmer()
    tweets_stemmed = []
    for tweet in tweets:
        tweet_stemmed = [lmt.lemmatize(w) for w in tweet]
        tweets_stemmed.append(tweet_stemmed)
    return tweets_stemmed

In [3]:
all_tweets = tokenize(data)
all_tweets = remove_url(all_tweets)
all_tweets = filter_noise(all_tweets)
all_tweets = remove_stopwords(all_tweets)
all_tweets = my_stem(all_tweets)



In [4]:
print(all_tweets)

[['please', 'see', 'link', 'youtube', 'channel', 'darteh', 'ministry', 'please', 'like', 'share', 'subscribe'], ['northeastedl', 'lol', 'happens', 'put', 'woman', 'charge'], ['side', 'man', 'get', 'smash', 'show', 'shout', 'tv', 'swear', 'get', 'ladddd'], ['le', '1', 'month'], ['dunzocs', 'lived', '3', 'year', 'visit', 'la', 'every', 'month', 'home', 'second', 'home', 'tbh', 'bday', 'could'], ['described', 'met', 'heart', 'broken', 'im', 'touched'], ['thetwomikes', 'dcrosss', 'talksport', 'talkradio', 'ya', 'dozy', 'sausage'], ['sometimes', 'life', 'hard', 'dust', 'keep', 'going', 'nolimit'], ['dunzocs', 'end', 'happening'], ['tmp', '03', 'c', 'wind', '0mph', 'press', '10274', 'mb', 'cloud', '837', 'ft', 'rain', '00', 'mm', 'humidity', '86', 'fine', 'weather', 'ukweather'], ['themissing', 'cmaawards50', 'wembley', 'spur', 'thistimenextyear', '201611', '3', '0114', 'gmt', 'trndnl'], ['tgfone4', 'worldseries', 'real', 'madrid', 'andy', 'carroll', 'sissoko', '201611', '3', '0114', 'gmt', 

The function create_tfidf_features  (input = array of tokenized tweets output = tf-idf feature matrix) Creates a distinct vocabulary set, and a dictionary (key val pairs)for each word in the vocabulary, calculated the IDF value, and for each term in each tweet, calculates the tfidf score (which is just the term frequency * IDF value)


https://nlpforhackers.io/tf-idf/

https://www.youtube.com/watch?v=4vT4fzjkGCQ&t=1s

http://billchambers.me/tutorials/2014/12/21/tf-idf-explained-in-python.html

In [14]:
def termfreq(word, tweet):
    #print("tweet:", doc, "length of tweet:", len(doc))
    num_words = len(tweet)
    word_occurences = tweet.count(word)
    return word_occurences/num_words

def count_doc_with_term(word, tweets):
    counter = 0
    for tweet in tweets:
        if word in tweet:
            counter = counter + 1
    return counter

#idf = amount_of_tweets/amount_of_tweets_with_word
def idf(word, tweets):
    num_tweets = len(tweets)
    num_tweets_with_word = count_doc_with_term(word, tweets)
    return np.log10(num_tweets/num_tweets_with_word)

def create_vocabulary(tweets):
    vocabulary = set() #creates a new set named vocabulary (set doesn't have duplicates)
    for tweet in tweets: #iterates over all tweets
        words = [w for w in tweet] #makes a list of words in a tweet (which our loop currently is in)
        vocabulary.update(words)#update teh vocab set 
    return list(vocabulary)

def compute_word_idf(tweets, vocabulary):
    word_idf = collections.defaultdict(lambda: 0)    #when I call x[k] for a nonexistent key k (such as a statement like v=x[k]), the key-value pair (k,0) will be automatically added to the dictionary, as if the statement x[k]=0 is first executed.  
    for w in vocabulary: #looping across each word in library to calculate the word's idf
        word_idf[w] = idf(w, tweets) #taking the wordidf dictionaries' key as the word and putting value as the idf
    return word_idf
    
def tf_idf(word, tweet, tweets, word_idf):
    if tweet != []:
        return termfreq(word, tweet) * word_idf[word]
    else:
        return 0
    
def tweet_tfidf_features(tweets):
    vocabulary = create_vocabulary(tweets)
    word_idf = compute_word_idf(tweets, vocabulary)
    tweet_features = np.empty(len(tweets), dtype=object)
    for i, tweet in enumerate(tweets):
        a = np.zeros(len(vocabulary))
        for j, word in enumerate(vocabulary):
            a[j] = tf_idf(word, tweet, tweets,word_idf)
        tweet_features[i] = a
        print(len(vocabulary))
    return np.vstack(tweet_features)

What is happening inside tweet_tfidf_features?

1) vocabulary = create_vocabulary(tweets) Creates a vocabulary set (non duplicate values in all words)
2) word_idf = returns a dictionary where key is the word and value is the word's idk


In [15]:
#test = [["this","a","a","is","sample"],["this","another","another","is","example","example","example"]]
#test2 = [["new","times","york"],["new","post","york"],["angeles","los","times"]]
feature_matrix = tweet_tfidf_features(all_tweets)

6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641
6641


In [11]:
np.set_printoptions(threshold=np.nan)

In [18]:
feature_matrix.shape

(2001, 6641)

In [19]:
def euclidean_vectorized(A, B):    
    n, d = A.shape
    m, d1 = B.shape
    assert d == d1, 'Incompatible shape'
    A_squared = np.sum(np.square(A), axis=1, keepdims=True)
    B_squared = np.sum(np.square(B), axis=1, keepdims=True)
    AB = np.matmul(A, B.T)
    distances = np.sqrt(A_squared - 2 * AB + B_squared.T)
    return distances

# X: data matrix of size (n_samples,n_features)
# n_clusters: number of clusters
# output 1: labels of X with size (n_samples,)
# output 2: centroids of clusters
def kmeans(X,n_clusters):
    # initialize labels and prev_labels. prev_labels will be compared with labels to check if the stopping condition
    # have been reached.
    prev_labels = np.zeros(X.shape[0])
    labels = np.zeros(X.shape[0])
    
    # init random indices
    # YOUR CODE GOES HERE
    indices = np.random.choice(X.shape[0], n_clusters, replace=False)# np.random.permutation(X.shape[0])[:n_clusters]
    
    # assign centroids using the indices
    # YOUR CODE GOES HERE
    centroids = X[indices]
    
    # the interative algorithm goes here
    while (True):
        # calculate the distances to the centroids
        # YOUR CODE GOES HERE
        distances = euclidean_vectorized(X,centroids)
        
        # assign labels
        # YOUR CODE GOES HERE
        labels = np.argmin(distances,axis=1)
        
        # stopping condition
        # YOUR CODE GOES HERE
        if np.array_equal(labels, prev_labels):
        #if np.sum(labels != prev_labels) == 0:
            break
        
        # calculate new centroids
        # YOUR CODE GOES HERE
        for cluster_indx in range(centroids.shape[0]):
            members = X[labels == cluster_indx]
            centroids[cluster_indx,:] = np.mean(members,axis=0)
        
        # keep the labels for next round's usage
        # YOUR CODE GOES HERE
        prev_labels = np.argmin(distances,axis=1)
    
    return labels,centroids
    

In [20]:
from sklearn.cluster import KMeans
num_clusters = 5
cluster_labels,centroids = kmeans(feature_matrix,num_clusters)

In [23]:
print(centroids)

[[  0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
    0.00000000e+00  

In [24]:
import sklearn
from sklearn.metrics import silhouette_score
print(silhouette_score(feature_matrix, cluster_labels))

-0.058117677133


In [ ]:
def cosine_distance(v1,v2):
    nomer = np.dot(v1, v2)
    denomer = np.dot(np.linalg.norm(v1), np.linalg.norm(v2))
    return nomer/denomer

In [7]:
#TODO from here
def calculate_distances(features):
    size = len(features)
    distances = np.zeros((size, size))
    for i in range(len(distances)):
        for j in range(len(distances)):
            distances[i,j] = cosine_distance(features[i], features[j])
    return distances

#distances = calculate_distances(feature_matrix)

In [55]:
from sklearn.metrics.pairwise import cosine_distances
from sklearn.metrics.pairwise import cosine_similarity